In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



In [7]:
df=pd.read_csv(r"C:\Users\Admin\Downloads\Telco-Customer-Churn.csv")

In [8]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [10]:
#------KNN CLASSIFIER-----
#changing the datatype from obj to int/float because ML models cannot handle object data type
df['TotalCharges'].replace(" ",0,inplace=True)
df['TotalCharges']=df['TotalCharges'].astype('float64')

In [11]:
#not required so dropping customer_id
df=df.drop('customerID',axis=1)

In [12]:
#mapping 0-->no and 1-->yes
df['SeniorCitizen']=df['SeniorCitizen'].map({0:'No',1:'Yes'})

In [13]:
#segregating the number columns and word columns
num_features=df.select_dtypes(include='number')
cat_features=df.select_dtypes(exclude='number')

In [14]:
#performing one hot encoding
cat_features_encoded = pd.get_dummies(data=cat_features, dtype=int)
churn_corr = cat_features_encoded.corr()['Churn_Yes'].drop(['Churn_Yes', 'Churn_No']).sort_values()

In [15]:

df_final=pd.get_dummies(data=df,drop_first=True,dtype=int)

In [16]:
df_final = df_final.drop(['gender_Male', 'PhoneService_Yes', 
                 'MultipleLines_No phone service',
                 'MultipleLines_Yes'], axis=1)

In [17]:
#this is the final modified dataframe after the pre processing is completed
df_final.head()

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen_Yes,Partner_Yes,Dependents_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,1,29.85,29.85,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,34,56.95,1889.50,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
2,2,53.85,108.15,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
3,45,42.30,1840.75,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,2,70.70,151.65,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [18]:
#there is a bias towards 0 over 1 so our model will not be as accurate.
df_final['Churn_Yes'].value_counts()

0    5174
1    1869
Name: Churn_Yes, dtype: int64

In [19]:
X=df_final.drop('Churn_Yes',axis=1)
y=df_final['Churn_Yes']

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [21]:
#using the standard scaler function for feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
#implementing the KNN classifier function in-built in the SKlearn library
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_jobs=-1,n_neighbors=5)
knn_model.fit(X_train,y_train)

KNeighborsClassifier(n_jobs=-1)

In [24]:
y_pred= knn_model.predict(X_test)

In [25]:
#developing the confusion matrix for our insights
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [26]:
confusion_matrix(y_test,y_pred)

array([[872, 161],
       [184, 192]], dtype=int64)

In [27]:
#acuracy of 75.51% is obtained using the KNN classifier algorithm
accuracy_score(y_test,y_pred)

0.7551454932576295

In [28]:
class_report=classification_report(y_test,y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.83      0.84      0.83      1033
           1       0.54      0.51      0.53       376

    accuracy                           0.76      1409
   macro avg       0.68      0.68      0.68      1409
weighted avg       0.75      0.76      0.75      1409



In [29]:
#-----LOGISTIC REGRESSION-----

#importing the logistic regression function
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()
reg.fit(X_train,y_train)

LogisticRegression()

In [30]:
y_pred = reg.predict(X_test)

In [31]:
confusion_matrix(y_test,y_pred)

array([[917, 116],
       [171, 205]], dtype=int64)

In [32]:
#as you can see, accuracy of 79.63% is obtained using logistic regression algorithm
accuracy_score(y_test,y_pred)

0.7963094393186657

In [33]:
class_report=classification_report(y_test,y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1033
           1       0.64      0.55      0.59       376

    accuracy                           0.80      1409
   macro avg       0.74      0.72      0.73      1409
weighted avg       0.79      0.80      0.79      1409



In [34]:
#-----NAIVE_BAYES CLASSIFIER-----
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB()

In [35]:
y_pred = classifier.predict(X_test)

In [36]:
confusion_matrix(y_test,y_pred)

array([[597, 436],
       [ 52, 324]], dtype=int64)

In [37]:
# naive bayes algorithm seems to give an accuracy of 65.36%
accuracy_score(y_test,y_pred)

0.6536550745209369

In [38]:
class_report=classification_report(y_test,y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.92      0.58      0.71      1033
           1       0.43      0.86      0.57       376

    accuracy                           0.65      1409
   macro avg       0.67      0.72      0.64      1409
weighted avg       0.79      0.65      0.67      1409



In [39]:
#--------DECISION TREES--------
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy')
tree.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [40]:
y_pred = tree.predict(X_test)

In [41]:
confusion_matrix(y_test,y_pred)

array([[854, 179],
       [185, 191]], dtype=int64)

In [42]:
#decision trees seems to give an accuracy score of 74.16% using ENTROPY method
accuracy_score(y_test,y_pred)

0.7416607523066004

In [43]:
class_report=classification_report(y_test,y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.82      0.83      0.82      1033
           1       0.52      0.51      0.51       376

    accuracy                           0.74      1409
   macro avg       0.67      0.67      0.67      1409
weighted avg       0.74      0.74      0.74      1409



In [48]:
tree2=DecisionTreeClassifier()
tree2.fit(X_train,y_train)

DecisionTreeClassifier()

In [49]:
y_pred=tree2.predict(X_test)

In [50]:
confusion_matrix(y_test,y_pred)

array([[846, 187],
       [185, 191]], dtype=int64)

In [51]:
#GINI seems to give a slightly lower accuracy of 73.59% in our particular example
accuracy_score(y_test,y_pred)

0.7359829666430092

In [52]:
class_report = classification_report(y_test, y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      1033
           1       0.51      0.51      0.51       376

    accuracy                           0.74      1409
   macro avg       0.66      0.66      0.66      1409
weighted avg       0.74      0.74      0.74      1409



In [53]:
#SUPPORT VECTOR MACHINES

#we can try with various kernels such as linear,polynomial,sigmoid and rbf
from sklearn.svm import SVC  
clf = SVC(kernel='linear')

In [54]:
clf.fit(X_train,y_train)

SVC(kernel='linear')

In [55]:
y_pred=clf.predict(X_test)

In [56]:
confusion_matrix(y_test,y_pred)

array([[905, 128],
       [180, 196]], dtype=int64)

In [57]:
#for linear kernel we get an accuracy score of 78.14% which is higher compared to other types
accuracy_score(y_test,y_pred)

0.7814052519517388

In [58]:
class_report = classification_report(y_test, y_pred)
print(class_report)

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      1033
           1       0.60      0.52      0.56       376

    accuracy                           0.78      1409
   macro avg       0.72      0.70      0.71      1409
weighted avg       0.77      0.78      0.78      1409

